<hr style="border:2px solid #0281c9"> </hr>

<img align="left" alt="ESO Logo" src="http://archive.eso.org/i/esologo.png">  

<div align="center">
  <h1 style="color: #0281c9; font-weight: bold;">ESO Science Archive</h1> 
  <h2 style="color: #0281c9; font-weight: bold;">Jupyter Notebooks</h2>
</div>

<hr style="border:2px solid #0281c9"> </hr>

# **ESO Phase-3 Multi-Instrument Overlap & Download Notebook**

In this example, we aim to find quasars that were observed with **UVES** or **XSHOOTER** (i.e., providing high-resolution 1D spectra) *and* also have been pointed at with **MUSE** (either specifically targeted or simply covered within its field of view).

This notebook demonstrates how to query and retrieve Phase-3 products from any two ESO instrument pairs (e.g. **MUSE** & **UVES** or **MUSE** & **XSHOOTER**) that spatially overlap around any sky position. Instead of manually inspecting tables and downloading files one-by-one, you’ll use:

* **ADQL/TAP** to discover overlapping products via the `query_instrument_overlap()` helper
* **Streamed HTTP + tqdm** to download each FITS file with a live progress bar via `download()`

Whether you’re studying quasars, galaxies, or any other targets with complementary integral-field and high-resolution spectroscopy, this notebook automates the entire end-to-end workflow—and can easily be extended to:

* Process dozens or hundreds of targets in batch (see example at end of notebook)
* Include additional instruments (e.g. HAWK-I, FORS2)
* Apply custom metadata filters (exposure time, wavelength range, etc.)

---

### 🗂️ **Data Products**

All products used in this example come from ESO’s Phase-3 public archive (see [Phase-3 overview](https://www.eso.org/sci/observing/phase3.html)):

- **MUSE** cubes: 3D data products delivering ∼1″ spatial sampling over a wide field  
- **UVES** spectra: high-resolution (R≳40,000) 1D spectra for precise kinematics & absorption  
- **XSHOOTER** spectra: medium-resolution (R∼4,000–17,000) cross-dispersed echelle covering 300–2,500 nm simultaneously, ideal for broad UV–NIR diagnostics  

---

### 🔍 **Query & Filtering**

Use `query_instrument_overlap(…)` to run an ADQL query against an ESO TAP endpoint. You can customize:

- **Position & radius** (`ra`, `dec`, `rad_deg`)  
- **Instrument names** (`instrument_primary`, `instrument_secondary`)  
- **Quality cuts** (`primary_abmaglim_min`, `secondary_snr_min`)  
- **TAP URL** and **`verbose`** printing  

The function returns an Astropy Table. 

---

### 🛠️ **Download Helpers**

We provide two functions:

1. **`download_single(dp_id, folder_path, …)`**  
   Streams a single DP-ID to disk with a **tqdm** progress bar.

2. **`download(dp_ids, folder_path, …)`**  
   Accepts either one DP-ID or a list, and returns:
   - **String** filename for a single ID  
   - **Dict** `{dp_id: filename or None}` for multiple  

---

### 💻 **How to Use This Notebook**

You can run it interactively via [Jupyter](https://jupyter.org/install) or view it as static HTML. This notebook (and others) is also hosted on the ESO programmatic access page:  
https://archive.eso.org/programmatic/#SCRIPT

- **Navigate** cells with the ↑/↓ keys  
- **Run** a cell with `Ctrl+Enter` (or `Cmd+Enter` on macOS)  
- **Restart & Run All** from the toolbar or `Kernel → Restart & Run All`  
- **Modify** query parameters (RA/Dec, instruments, filters) and re-execute to explore different targets  

<hr style="border:2px solid #0281c9">

## **Importing the necessary modules**

In [1]:
# Standard library
import cgi
import os

# Third-party dependencies
import numpy as np
import requests
from tqdm import tqdm

# Astronomy-specific packages
from astropy import units as u
from astropy.coordinates import SkyCoord
from pyvo.dal import TAPService

/var/folders/3y/cmsgt3_x7x72bbs__zhy_q6h0008wr/T/ipykernel_61148/1907603823.py:2: DeprecationWarning: 'cgi' is deprecated and slated for removal in Python 3.13
  import cgi


## **Define some useful functions** 
Let's define a couple of utility functions to first find the dataset pairs, and a useful function to write the files on disk.

In [2]:
def query_instrument_overlap(
    ra, dec, rad_deg,
    instrument_primary='MUSE',
    instrument_secondary='UVES',
    primary_abmaglim_min=None,
    secondary_snr_min=None,
    tap_url='https://archive.eso.org/tap_obs',
    verbose=False
):
    """
    Query ESO TAP for overlapping Phase-3 products of two instruments around a sky position,
    with optional AB-magnitude and SNR filters.

    **Note:**  
    - Applying `primary_abmaglim_min` requires that the primary instrument provides an
      `abmaglim` field (e.g. imaging or cube products); otherwise the result set will be empty.  
    - Applying `secondary_snr_min` requires that the secondary instrument is a spectrograph
      with an `snr` metadata field.

    Parameters
    ----------
    ra, dec : float
        Target coordinates in decimal degrees.
    rad_deg : float
        Search radius in degrees.
    instrument_primary : str, optional
        Name of the primary instrument (default: 'MUSE').
    instrument_secondary : str, optional
        Name of the secondary instrument (default: 'UVES').
    primary_abmaglim_min : float or None
        Minimum AB magnitude limit for primary instrument.
    secondary_snr_min : float or None
        Minimum SNR for secondary instrument.
    tap_url : str, optional
        URL of the TAP service.
    verbose : bool, optional
        If True, print the ADQL query.

    Returns
    -------
    astropy.table.Table
        Table with columns ['{primary.lower()}_id', 'abmaglim', '{secondary.lower()}_id', 'snr'].
    """
    # Build filter clauses
    extra = []
    if primary_abmaglim_min is not None:
        extra.append(f"PRI.abmaglim > {primary_abmaglim_min}")
    if secondary_snr_min is not None:
        extra.append(f"SEC.snr > {secondary_snr_min}")
    extra_where = ("\n  AND " + "\n  AND ".join(extra)) if extra else ""

    # Aliases
    PRI, SEC = instrument_primary, instrument_secondary
    pri_alias = 'PRI'
    sec_alias = 'SEC'

    # ADQL query
    query = f"""
    SELECT DISTINCT
      {pri_alias}.dp_id          AS {instrument_primary.lower()}_id,
      {pri_alias}.abmaglim       AS {instrument_primary.lower()}_abmaglim,
      {pri_alias}.access_estsize AS {instrument_primary.lower()}_size,
      {sec_alias}.dp_id          AS {instrument_secondary.lower()}_id,
      {sec_alias}.snr            AS {instrument_secondary.lower()}_snr,
      {sec_alias}.access_estsize AS {instrument_secondary.lower()}_size
    FROM
      (SELECT * FROM ivoa.Obscore WHERE instrument_name = '{instrument_secondary}') {sec_alias},
      (SELECT * FROM ivoa.Obscore WHERE instrument_name = '{instrument_primary}') {pri_alias}
    WHERE
      INTERSECTS({sec_alias}.s_region, {pri_alias}.s_region) = 1
      AND INTERSECTS(
        {sec_alias}.s_region,
        CIRCLE('', {ra}, {dec}, {rad_deg})
      ) = 1
    {extra_where}
    """

    if verbose:
        print(query)

    tapobs = TAPService(tap_url) # For the ESO TAP service
    results = tapobs.search(query=query) #Maximum records to return

    return results.to_table()

def getDispositionFilename(response):
    """
    Get the filename from the Content-Disposition header of an HTTP response.

    Parameters
    ----------
    response : requests.Response
        The HTTP response object.

    Returns
    -------
    str or None
        The filename parsed from the Content-Disposition header, or None if not present.
    """
    content_disposition = response.headers.get('Content-Disposition')
    if content_disposition is None:
        return None
    _, params = cgi.parse_header(content_disposition)
    return params.get('filename')


def download_single(dp_id, folder_path="./", timeout=600, block_size=1024):
    """
    Download a file given its DP-ID from the ESO archive, saving it to a folder with a progress bar.

    Parameters
    ----------
    dp_id : str
        The DP-ID of the ESO product to download.
    folder_path : str, optional
        Directory where files will be saved (default: current directory).
    timeout : int, optional
        HTTP timeout in seconds (default: 600).
    block_size : int, optional
        Number of bytes per chunk when streaming (default: 1024).

    Returns
    -------
    str
        The filename of the downloaded file.

    Raises
    ------
    RuntimeError
        If the HTTP response status is not 200 or no filename header is present.
    """
    base_url = "https://dataportal.eso.org/dataportal_new/file/"
    access_url = f"{base_url}{dp_id}"

    # Stream the GET request
    response = requests.get(access_url, stream=True, timeout=timeout)
    if response.status_code != 200:
        raise RuntimeError(f"Download failed for {dp_id}: status {response.status_code}")

    # Parse filename
    filename = getDispositionFilename(response)
    if not filename:
        raise RuntimeError(f"No filename in headers for {dp_id}")
    full_path = os.path.join(folder_path, filename)

    # Total size
    total_size = int(response.headers.get('Content-Length', 0))

    # Write with tqdm
    with open(full_path, 'wb') as f, tqdm(
        total=total_size,
        unit='iB',
        unit_scale=True,
        desc=f"Downloading {dp_id}"
    ) as bar:
        for chunk in response.iter_content(chunk_size=block_size):
            if chunk:
                f.write(chunk)
                bar.update(len(chunk))

    return filename


def query_dp_ids(dp_id, tap_url='https://archive.eso.org/tap_obs', verbose=False):
    """
    Query the ESO TAP service for a specific DP-ID and return the result as an astropy table.

    Parameters
    ----------
    dp_id : str
        The DP-ID to query.
    tap_url : str, optional
        URL of the TAP service (default: 'https://archive.eso.org/tap_obs').
    verbose : bool, optional
        If True, print the ADQL query.

    Returns
    -------
    astropy.table.Table
        Table with columns ['dp_id', 'access_url', 'access_estsize'].
    """

    if isinstance(dp_id, list) or isinstance(dp_id, np.ndarray):

        in_clause = ",".join(f"'{i}'" for i in dp_id)
        query = f"""
        SELECT dp_id, access_url, access_estsize
        FROM ivoa.Obscore
        WHERE dp_id IN ({in_clause})
        """

    elif isinstance(dp_id, str):

        query = f"""
        SELECT dp_id, access_url, access_estsize
        FROM ivoa.Obscore
        WHERE dp_id = '{dp_id}'
        """

    if verbose:
        print(query)

    tapobs = TAPService(tap_url)
    results = tapobs.search(query=query)

    return results.to_table()

def download(dp_ids, folder_path="./", timeout=600, block_size=1024):
    """
    Download one or multiple ESO products given DP-ID(s), saving with a progress bar.

    Parameters
    ----------
    dp_ids : str or list of str
        Single DP-ID or list of DP-IDs to download.
    folder_path : str, optional
        Directory where files will be saved (default: current directory).
    timeout : int, optional
        HTTP timeout in seconds for each download.
    block_size : int, optional
        Number of bytes per chunk when streaming.

    Returns
    -------
    str or dict
        If a single DP-ID is provided, returns the filename (str).
        If multiple, returns a dict mapping each DP-ID to its downloaded filename or None on failure.
    """

    # Ensure folder exists
    os.makedirs(folder_path, exist_ok=True)

    # Remove douplicate DP-IDs
    dp_ids = np.unique(dp_ids)
    
    # Normalize input
    if isinstance(dp_ids, str):
        dp_ids = [dp_ids]
    else: 
        try: 
            dp_ids = dp_ids.tolist() 
        except: 
            dp_ids = list(dp_ids) 

    # Check space of downloads
    total_space = 0
    result = query_dp_ids(dp_ids)
    total_space = np.sum(result['access_estsize']) 

    # Check free space on machine
    def get_free_bytes_statvfs(path: str):
        """Return free space (in bytes) on Unix-like systems using statvfs."""
        st = os.statvfs(path)
        return st.f_bavail * st.f_frsize
    free_space = get_free_bytes_statvfs(folder_path)

    print(f"Total download size: \t {total_space / (1024):.2f} MiB")
    print(f"Free space available: \t {free_space / (1024 * 1024):.2f} MiB")

    if total_space > free_space:
        raise RuntimeError(
            f"Not enough space to download {len(dp_ids)} files. "
            f"Required: {total_space / (1024):.2f} MiB, "
            f"Available: {free_space / (1024 * 1024):.2f} MiB."
        )
    else:
        print("   Sufficient space available for downloads.")

    # Start downloading...
    results = {}
    for dp_id in dp_ids:
        try:
            fname = download_single(dp_id, folder_path, timeout, block_size)
            results[dp_id] = fname
        except Exception as e:
            print(f"Failed to download {dp_id}: {e}")
            results[dp_id] = None

    print(f"Downloaded {len(results)} files to {folder_path}")
    
    return 

## **Query and Download data** 
Now we can query the ESO TAP service for MUSE and UVES data, and download the results.

First we need to define the coordinates of the target, and the radius around it to search for data. Here we use "**QSO J1538+0855**" and "**QSO B1725-142**" as examples, but you can replace it with any other target name or coordinates. We make use of the `SkyCoord` class from `astropy.coordinates` to handle resolving the coordinates of both targets and converting them to degrees. 

### **MUSE vs UVES query**  
First, we query for where we have both **MUSE** and **UVES** data available for the same target. Note that here we provide a constraint on the `snr` of the **UVES** data, to ensure we only get high-quality data. Moreover, we show the file sizes in megabytes (MB) for easy reference if your machine as sufficient space to download. 

> **Note:** Use of `primary_abmaglim_min` and `secondary_snr_min`... 
> - Applying an AB-magnitude filter (`primary_abmaglim_min`) requires that the primary instrument (MUSE) provides an `abmaglim` field (i.e. cubes or imaging products).  
> - Applying an SNR filter (`secondary_snr_min`) requires that the secondary instrument (UVES) produces spectra with an `snr` metadata field.  

> **Tip:** The output table lists every possible MUSE–UVES pairing. If there are _N_ MUSE products and _M_ UVES products, you’ll see _N × M_ rows (i.e. duplicated entries). To reduce this to unique files, simply filter by `muse_id` or `uves_id`, or use `np.unique()` on the DP-IDs.  

In [3]:
target_muse_uves = "QSO J1538+0855"
coord = SkyCoord.from_name(target_muse_uves)
ra = coord.ra.deg # Right Ascension in degrees
dec = coord.dec.deg # Declination in degrees
radius = (5*u.arcsec).to("deg").value # Radius in degrees

table_muse_uves = query_instrument_overlap(ra, dec, radius, instrument_primary='MUSE', instrument_secondary='UVES', secondary_snr_min=2.0)

print(f"Number of unique MUSE files: {len(np.unique(table_muse_uves['muse_id']))}")
print(f"Number of unique UVES files: {len(np.unique(table_muse_uves['uves_id']))}")

print(f"MUSE FILES:")
for file in np.unique(table_muse_uves['muse_id']):
    size = table_muse_uves[table_muse_uves['muse_id'] == file]['muse_size'][0]*u.kbyte.to("MB")
    print(f"   {file} ({size:.2f} MB)")
print(f"UVES FILES:")
for file in np.unique(table_muse_uves['uves_id']):
    size = table_muse_uves[table_muse_uves['uves_id'] == file]['uves_size'][0]*u.kbyte.to("MB")
    print(f"   {file} ({size:.2f} MB)")

Number of unique MUSE files: 9
Number of unique UVES files: 2
MUSE FILES:
   ADP.2017-09-19T14:57:26.141 (3404.19 MB)
   ADP.2024-05-06T15:04:25.542 (5534.09 MB)
   ADP.2024-05-14T00:58:53.084 (5585.09 MB)
   ADP.2024-06-14T15:24:37.101 (5610.59 MB)
   ADP.2024-06-14T15:24:37.107 (5585.09 MB)
   ADP.2024-07-11T02:05:18.904 (5597.63 MB)
   ADP.2024-07-11T02:05:18.910 (5598.10 MB)
   ADP.2024-07-11T02:05:18.916 (5623.78 MB)
   ADP.2024-12-09T14:21:42.814 (6287.37 MB)
UVES FILES:
   ADP.2023-04-24T10:24:40.344 (1.97 MB)
   ADP.2023-04-24T10:24:40.348 (1.97 MB)


### **MUSE vs XSHOOTER query** 
Second, we then query for where we have both **MUSE** and **XSHOOTER** data available for the same target. Again, we provide a constraint on the SNR of the XSHOOTER data... 

In [4]:
target_muse_xshooter = "QSO B1725-142"
coord = SkyCoord.from_name(target_muse_xshooter)
ra = coord.ra.deg # Right Ascension in degrees
dec = coord.dec.deg # Declination in degrees
radius = (5*u.arcsec).to("deg").value # Radius in degrees

table_muse_xshooter = query_instrument_overlap(ra, dec, radius, instrument_primary='MUSE', instrument_secondary='XSHOOTER', secondary_snr_min=2.0)

print(f"Number of unique MUSE files: {len(np.unique(table_muse_xshooter['muse_id']))}")
print(f"Number of unique XSHOOTER files: {len(np.unique(table_muse_xshooter['xshooter_id']))}")

print(f"MUSE FILES:")
for file in np.unique(table_muse_xshooter['muse_id']):
    size = table_muse_xshooter[table_muse_xshooter['muse_id'] == file]['muse_size'][0]*u.kbyte.to("MB")
    print(f"   {file} ({size:.2f} MB)")
print(f"XSHOOTER FILES:")
for file in np.unique(table_muse_xshooter['xshooter_id']):
    size = table_muse_xshooter[table_muse_xshooter['xshooter_id'] == file]['xshooter_size'][0]*u.kbyte.to("MB")
    print(f"   {file} ({size:.2f} MB)")

Number of unique MUSE files: 4
Number of unique XSHOOTER files: 144
MUSE FILES:
   ADP.2019-07-19T00:01:04.352 (3901.63 MB)
   ADP.2019-07-24T09:10:32.260 (3414.35 MB)
   ADP.2019-09-18T17:07:14.428 (3913.50 MB)
   ADP.2019-09-20T03:41:52.861 (3935.12 MB)
XSHOOTER FILES:
   ADP.2023-04-20T08:25:01.746 (1.36 MB)
   ADP.2023-04-20T08:25:01.799 (0.74 MB)
   ADP.2023-04-20T08:25:01.842 (1.33 MB)
   ADP.2023-05-11T09:34:46.825 (0.74 MB)
   ADP.2023-05-11T09:34:46.837 (1.33 MB)
   ADP.2023-05-11T09:34:46.877 (1.36 MB)
   ADP.2023-06-01T07:51:00.477 (1.33 MB)
   ADP.2023-06-01T07:51:00.697 (0.74 MB)
   ADP.2023-06-01T07:51:00.726 (1.36 MB)
   ADP.2023-06-01T08:07:05.821 (1.36 MB)
   ADP.2023-06-01T08:07:05.866 (1.33 MB)
   ADP.2023-06-01T08:07:05.876 (0.74 MB)
   ADP.2023-06-05T08:59:45.004 (1.36 MB)
   ADP.2023-06-05T08:59:45.021 (1.33 MB)
   ADP.2023-06-05T08:59:45.144 (0.74 MB)
   ADP.2023-06-15T06:19:08.130 (1.34 MB)
   ADP.2023-06-15T06:19:08.156 (0.74 MB)
   ADP.2023-06-15T06:19:08.160 

## **Download MUSE, UVES, and XSHOOTER data** 
Finally, we can download the MUSE, UVES, and XSHOOTER data for the targets we found in the previous steps. Here we output the files to a folder named after the instrument combination and target - e.g. ``./data/muse_uves_QSO J1538+0855/``. If this directory does not exist, it will be created automatically. 

**Note:** here we only download the a subset of the results, for demonstration purposes.

In [5]:
# download(table_muse_uves["muse_id"][0], folder_path=f"./data/muse_uves_{target_muse_uves}/")
download(table_muse_uves["uves_id"], folder_path=f"./data/muse_uves_{target_muse_uves}/")

# download(table_muse_xshooter["muse_id"][0], folder_path=f"./data/muse_xshooter_{target_muse_xshooter}/")
# download(table_muse_xshooter["xshooter_id"], folder_path=f"./data/muse_xshooter_{target_muse_xshooter}/")

Total download size: 	 3.85 MiB
Free space available: 	 246977.39 MiB
   Sufficient space available for downloads.


Downloaded 2 files to ./data/muse_uves_QSO J1538+0855/


## Aladin Lite Preview

The `show_aladin` helper function embeds **four** synchronized Aladin Lite viewers — each occupying 25% of the container width — so you can compare the same sky region across multiple surveys at once. These are also locked to the same FOV such that zooming or moving in panel moves the rest of the panels. Simply call:

```python
show_aladin(target="YOUR_TARGET_NAME")
```

**Surveys Displayed**

| Panel | Survey ID                             | Description                                  |
|-------|---------------------------------------|----------------------------------------------|
| **A** | `P/DSS2/color`                        | DSS2 color composite (blue / red / IR plates) |
| **B** | `P/SDSS9/color-alt`                   | SDSS DR9 color-alt (u / g / r bands)         |
| **C** | `P/PanSTARRS/DR1/color-i-r-g`         | Pan-STARRS DR1 (i / r / g bands)             |
| **D** | `P/PanSTARRS/DR1/color-z-zg-g`        | Pan-STARRS DR1 (z / z₉ / g bands)            |


**Key Features**

- **Synchronized Target**  
  Panning any viewer recenters **all** others on the same coordinates.
- **Synchronized FoV**  
  Zooming in or out in one panel updates the field-of-view across all four.
- **Minimal UI**  
  Projection and fullscreen controls are hidden for a cleaner display.

In [6]:
def show_aladin(target, fov=0.025): 
    from ipyaladin import Aladin
    from ipywidgets import Layout, Box, widgets

    cosmetic_options = {"show_projection_control": False, "show_fullscreen_control": False}

    a = Aladin(layout=Layout(width="25%"), target=target, survey="P/DSS2/color",fov=fov, **cosmetic_options)
    b = Aladin(layout=Layout(width="25%"), survey="P/SDSS9/color-alt",  fov=fov, **cosmetic_options)
    c = Aladin(layout=Layout(width="25%"), survey="P/PanSTARRS/DR1/color-i-r-g", fov=fov, **cosmetic_options)
    d = Aladin(layout=Layout(width="25%"), survey="P/PanSTARRS/DR1/color-z-zg-g", fov=fov, **cosmetic_options)

    # synchronize target between 3 widgets
    widgets.jslink((a, "_target"), (b, "_target"))
    widgets.jslink((b, "_target"), (c, "_target"))
    widgets.jslink((c, "_target"), (d, "_target"))

    # synchronize FoV (zoom level) between 3 widgets
    widgets.jslink((a, "_fov"), (b, "_fov"))
    widgets.jslink((b, "_fov"), (c, "_fov"))
    widgets.jslink((c, "_fov"), (d, "_fov"))

    items = [a, b, c, d]

    box_layout = Layout(display="flex", flex_flow="row", align_items="stretch", border="solid", width="100%")
    box = Box(children=items, layout=box_layout)
    return box

### **QSO J1538+0855**

In [7]:
show_aladin(target_muse_uves)

Box(children=(Aladin(layout=Layout(width='25%'), survey='P/DSS2/color'), Aladin(layout=Layout(width='25%'), su…

### **QSO B1725-142**

In [8]:
show_aladin(target_muse_xshooter)

Box(children=(Aladin(layout=Layout(width='25%'), survey='P/DSS2/color'), Aladin(layout=Layout(width='25%'), su…

## **End-to-end example for large target lists**
Here we provide an example of how to use the `query_instrument_overlap()` and `download()` functions in a loop to process multiple targets at once. This is useful if you have a list of targets and want to download data for each one without manually repeating the steps. Note that here we provide a fixed search radius of 5 arcsec. 

In [9]:
# target_list = ["QSO J1538+0855", "QSO B1725-142"]
# radius = (5*u.arcsec).to("deg").value 

# for target in target_list:

#     print(f"Processing target: {target}")

#     coord = SkyCoord.from_name(target)
#     ra = coord.ra.deg
#     dec = coord.dec.deg
#     radius = (5*u.arcsec).to("deg").value

#     table_muse_uves = query_instrument_overlap(ra, dec, radius, instrument_primary='MUSE', instrument_secondary='UVES', secondary_snr_min=2.0)
#     table_muse_xshooter = query_instrument_overlap(ra, dec, radius, instrument_primary='MUSE', instrument_secondary='XSHOOTER', secondary_snr_min=2.0)

#     download(table_muse_uves["muse_id"], folder_path=f"./data/muse_uves_{target}/")
#     download(table_muse_uves["uves_id"], folder_path=f"./data/muse_uves_{target}/")
#     download(table_muse_xshooter["muse_id"], folder_path=f"./data/muse_xshooter_{target}/")
#     download(table_muse_xshooter["xshooter_id"], folder_path=f"./data/muse_xshooter_{target}/")